# CV란?

컴퓨터 비전이란 컴퓨터가 세계를 시각적인 측면에서 이해할 수 있도록 학습시키는 인공지능 분야이다. 기본적으로 크게 4가지의 테스크로 나눌 수 있는데, 

1. **Image Classification**
2. **Object Detection**
3. **Semantic Segmentation**
4. **Instance Segmentation**

로 나뉜다.

<img src="http://www.smart-interaction.com/wp/wp-content/uploads/2022/06/71-768x768.png" alt="Alternative text" style="width:400px;" />

* Image Classification은 사진을 분석해서 사진에 있는 물체가 어느 종류에 속하는지 분류하는 Task이다.
<br>
* Object detection은 사진을 분석해 사진 안에 존재하는 복수의 물체에 대해서 각각을 분류함과 동시에 사진에서의 위치를 판별하는 Task이다.
<br>
* Semantic Segmentation은 사진 내에서 같은 종류에 속하는 물체의 면적을 정확하게 예측하는 Task이다.
<br>
* 마지막으로 Instance Segmentation은 Object Detection과 Semantic Segmentation을 합친 것으로, 사진 내에서 같은 종류라 할지라도 개별적인 물체들을 서로 독립적으로 분류하는 Task이다.

<img src="https://www.anolytics.ai/wp-content/uploads/2022/07/segment_sgment.jpg" alt="Alternative text" style="width:400px;" />

ps: Semantic Segmentation에서도 다른 종류들은 다른 색깔로 동시에 분류할 수 있다.

매우 복잡하고 정교한 모델처럼 보이지만 그 근간은 아주 간단한 인공지능 레이어가 촘촘히 쌓이고 학습되어 만들어진 것이다. 즉, 기본을 배우면서 분석한다면 충분히 분석하고 만들어 낼 수 있다.

# CV Basic

## Linear layer의 한계

예전에 튜토리얼로 수행했던 MNIST 예제를 기억할 것이다. MNIST를 분석하기 위해서 우리는 받은 (1x28x28)짜리의 MNIST사진을 Flatten시키고 복수의 Linear층과의 행렬연산을 통해서 값을 도출했다.

MNIST처럼 간단한 예제에서는 그 값을 매우 정확하게 예측할 수 있었다.

하지만 Linear 층만으로 입체적인 이미지 데이터를 분석하기에는 한계가 존재한다. 아래의 예제를 보자.

In [ ]:
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torchsummary import summary as summary_


transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 64

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


CIFAR10 이미지 데이터셋을 DNN을 이용한 인공지능으로 학습하고자 한다.

In [ ]:
class FlatModel(nn.Module):
    def __init__(self):
        super(FlatModel, self).__init__()
        self.flat = nn.Flatten()

        self.Linear1 = nn.Linear(3*32*32, 400)
        self.relu1 = nn.ReLU()
        self.Linear2 = nn.Linear(400, 500)
        self.relu2 = nn.ReLU()
        self.Linear3 = nn.Linear(500,10)
        self.softmax = nn.Softmax(dim=1)
        
        
        
    def forward(self, x):
        x = self.flat(x)
        
        out = self.Linear1(x)
        out = self.relu1(out)
        out = self.Linear2(out)
        out = self.relu2(out)
        out = self.Linear3(out)
        out = self.softmax(out)
        
        return out

flatModel = FlatModel()
flatModel = flatModel.cuda()

summary_(flatModel, (3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                 [-1, 3072]               0
            Linear-2                  [-1, 400]       1,229,200
              ReLU-3                  [-1, 400]               0
            Linear-4                  [-1, 500]         200,500
              ReLU-5                  [-1, 500]               0
            Linear-6                   [-1, 10]           5,010
           Softmax-7                   [-1, 10]               0
Total params: 1,434,710
Trainable params: 1,434,710
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.04
Params size (MB): 5.47
Estimated Total Size (MB): 5.52
----------------------------------------------------------------


벌써 파라미터의 수가 매우 많은 것을 확인 할 수 있다.

In [ ]:
import torch.optim as optim


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(flatModel.parameters(), lr=0.01)


def train(model, epoch, device):
    model.train()
    
    for i in range(epoch):
        train_loss = 0
        correct = 0
        total = 0

        for idx, (input, target) in enumerate(trainloader):
            
            
            input = input.to(device)
            target = target.to(device)
            output = model(input)
            loss = criterion(output, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

        acc = 100 * correct / total
        print('train epoch : {} [{}/{}]| loss: {:.3f} | acc: {:.3f}'.format(
        i, idx, len(trainloader), train_loss/(idx+1), acc))
    
    print("end of training")
    

train(flatModel, 10, 'cuda')

train epoch : 0 [781/782]| loss: 2.299 | acc: 17.542
train epoch : 1 [781/782]| loss: 2.282 | acc: 19.534
train epoch : 2 [781/782]| loss: 2.243 | acc: 21.696
train epoch : 3 [781/782]| loss: 2.212 | acc: 23.462
train epoch : 4 [781/782]| loss: 2.195 | acc: 25.326
train epoch : 5 [781/782]| loss: 2.180 | acc: 27.430
train epoch : 6 [781/782]| loss: 2.166 | acc: 29.338
train epoch : 7 [781/782]| loss: 2.153 | acc: 30.996
train epoch : 8 [781/782]| loss: 2.140 | acc: 32.442
train epoch : 9 [781/782]| loss: 2.126 | acc: 34.016
end of training


Linear레이어를 이용한 간단한 모델을 제작해서 10번의 에폭을 거쳤음에도 눈에 띄는 변화가 일어나지 않았다. 1차원 흑백 사진이었던 MNIST와는 다르게 CIFAR10은 컬러 이미지로 3x32x32의 고차원 정보를 담고 있어 Linear 레이어 만으로 모든 정보를 제대로 학습하기가 쉽지 않아서도 있지만, Linear레이어의 구조적인 문제점 또한 존재한다. CIFAR10 데이터를 활용할 때 기존의 Linear 방식의 문제점을 알 수 있는데, 바로 컬러 이미지를 위치, 색깔에 관계없이 무조건 Flatten을 한 뒤 완전 연결시켜 행렬연산을 한다는 것이다.

<img src="https://cdn.crowdpic.net/detail-thumb/thumb_d_EFB9A72049730D626BB3A7CFF4C08AF5.jpeg" alt="Alternative text" style="width:300px;" />


우리는 사물을 인식할 때 해당 사물의 경계(색 변화)나 사물의 모양새 등 공간적 정보를 이용한다. 하지만 이와 다르게 Linear 층은 모든 경계를 무너뜨리고, 각 구역별 연산도 수행하지 않아 이러한 공간적 정보를 활용하지 못한다.

공간적 정보를 인공지능에게 학습하기 위해 인간의 눈을 모방해서 구조가 바로 우리가 오늘 배울 CNN(Convolution Neural Network)이다.

## CNN에 대해서

CNN에 대해서 간단하게 이해해보자. CNN에 대한 설명은 정말로 많이 있으니 여기서 이해가 안 돼도 인터넷을 찾아보도록하자.
<br>
참고로 필자는 쉽게 이해하려면 https://youngq.tistory.com/40 를 추천한다.

CNN의 구조는 크게 아래의 구조를 띈다.

<img src="https://taewanmerepo.github.io/2018/01/cnn/head.png" alt="Alternative text" style="width:300ㅔx;" />


Input을 이용해서 공간적 정보들을 추출, 압축하는 Feature extraction 단계와 그 정보들을 이용하는 우리가 일반적으로 DNN에서 수행하는 Classification 단계가 있다.

### Feature extraction

Feature extraction은 공간적 정보를 압축하기 위해 Convolution 계층과 Pooling 계층을 이용한다.

<img src="https://ifh.cc/g/N5dJmg.png" alt="Alternative text" width="250"/>

이 숫자 2가 담긴 이미지를 input된 이미지라고 하자. 이 사진은 겉으로 보기에는 연속된 선으로 이루어진 숫자처럼 보이지만 사실 매우 작은 픽셀단위로 나뉘어져 있으며, 각 픽셀에는 밝기를 나타내는 값이 저장되어 있다.

<img src="https://ifh.cc/g/gvCSm5.png" alt="Alternative text" width="300"/>

<img src="https://ifh.cc/g/g0CQMz.jpg" alt="Alternative text" width="300"/>



Convolution은 이러한 공간적으로 밀접한 픽셀들의 정보를 더 잘 함축하기 위해 고안된 계층으로 구역(kernel)별로 나누어서 해당 구역 내에 같이 존재하는 픽셀별로 연산을 수행한다.

<img src="https://taewanmerepo.github.io/2018/01/cnn/conv.png" alt="Alternative text" width="500"/>

Kernel_size 만큼의 크기로 정의된 Filter가 Input 위를 움직이며 해당 구역에 포함된 픽셀들을 필터와 곱한 다음 합하게 되고, 그 값을 output에 해당하는 Feature Map에 저장하게 된다

<img src="https://taewanmerepo.github.io/2018/01/cnn/filter.jpg" alt="Alternative text" width="500"/>


이때, Filter의 수는 여러 개일 수 있으며, 각 필터들의 연산으로 나온 값은 각각 다른 Feature Map에 저장된다.
<br>
즉, Filter의 수만큼 output인 Feature Map의 수가 정의된다는 것이다

<img src="https://ifh.cc/g/6k0ofS.jpg" alt="Alternative text" width="700"/>

그런데 위에 사진에서 처럼 kernel_size로 인해서 Feature Map의 크기는 항상 원본인 Input보다 작다는 것을 알 수 있다. 이는 Convolution층의 문제인데, 바로 층을 깊게 만들수록 Feature Map이 점점 작아져 일반적인 방법으로는 층의 깊이를 키우는데 한계가 있다는 것이다.
<br>
그래서 Convolution layer에서는 이를 해결하기 위해서 **Padding**이라는 것을 사용한다

<img src="https://taewanmerepo.github.io/2018/01/cnn/padding.png" alt="Alternative text" width="400"/>

이렇게 input data의 가장자리에 0을 추가해서 이미지를 실제보다 크게 만드는 것이다. 이렇게 한다면 kernel_size로 인해서 감소하는 Feature Map의 크기와 padding으로 인해 늘어난 Feature Map의 크기가 상쇄되어 Input과 Feature Map의 크기를 일정하게 유지할 수 있다.

하지만, 패딩을 항상 사용하지는 않는다. 맨 위 CNN의 구조에서 볼 수 있듯이, CNN은 공간적 정보를 압축하며 Map의 크기를 줄이는 대신, Channel의 수를 늘려 정보를 저장한다. 그래서 의도적으로 Map의 크기를 줄이기도 한다.
<br>
이때, CNN 같은 가중치를 이용한 압축이 아닌, Pooling을 통한 압축을 수행할 때도 있다.

<img src="https://taewanmerepo.github.io/2018/02/cnn/maxpulling.png" alt="Alternative text" width="400"/>

이처럼 구역 내의 최댓값 또는 평균을 통해 새로운 Map을 만들어 압축하는 경우가 있다.
<br>
**이때, Pooling layer에서는 feature map의 증가는 일어나지 않는다**. 단순히 크기를 줄일 뿐이다.

### CNN의 구조

<img src="https://miro.medium.com/v2/resize:fit:828/format:webp/1*uAeANQIOQPqWZnnuH-VEyw.jpeg" alt="Alternative text" width="600" />

다시 해당 CNN의 구조를 보면 Convolution layer를 거치면서 feature map의 크기가 줄어들고 그 수는 늘어났고, Pooling 과정에서는 크기만 줄어드는 것을 확인할 수 있다.
<br>
이렇게 압축을 거치면서 인공지능은 이미지가 갖는 공간적 정보(선, 원, 경계 등)에 대해서 학습하며 DNN보다 효과적으로 학습을 할 수 있는 것이다.

### Convolution pytorch documentation 살펴보기

https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
<br>
해당 문서는 covolution layer에 해당하는 torch.nn.Conv2d 클래스에 대해서 설명하고 있다. 읽어본다면 직접 코딩할 때 많은 도움이 될 것이다. 

<p>1. stride : Filter가 한번에 움직이는 길이이다. 일반적으로는 1로 설정되어 있지만 여러 목적으로 더 커질 수 있다. 해당 예시에서는 2</p>
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fbn02Ql%2Fbtq1Nvc2Ywd%2FKEzmwiYmIOZSYSj2ESb4ZK%2Fimg.png" alt="Alternative text" width="400" />
<br>
<p>2. in_channels, out_channels : input의 map 수와 output인 feature map의 수(Filter의 수)를 정의하는 부분</p>
<br>
<p>3. kernel_size : Filter의 크기</p>



## CNN 구조 코딩

동일한 CIFAR10 데이터셋을 학습시켜 CNN 모델의 강력함을 알아보자.

In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 6, kernel_size=3, stride=2, padding = 1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(6, 6, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(6, 12, kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(12, 12, kernel_size=3, stride=1,padding=1)

        self.flat = nn.Flatten()
        self.dense1 = nn.Linear(8*8*12, 100)
        self.dense2 = nn.Linear(100,10)
        
    def forward(self, x):
        
        out = self.conv1(x)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.relu(out)
        out = self.conv4(out)

        out = self.flat(out)
        out = self.dense1(out)
        out = self.dense2(out)


        return out

NewModel = CNNModel()
NewModel = NewModel.cuda()

summary_(NewModel, (3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 16, 16]             168
              ReLU-2            [-1, 6, 16, 16]               0
            Conv2d-3            [-1, 6, 16, 16]             330
              ReLU-4            [-1, 6, 16, 16]               0
            Conv2d-5             [-1, 12, 8, 8]             660
              ReLU-6             [-1, 12, 8, 8]               0
            Conv2d-7             [-1, 12, 8, 8]           1,308
           Flatten-8                  [-1, 768]               0
            Linear-9                  [-1, 100]          76,900
           Linear-10                   [-1, 10]           1,010
Total params: 80,376
Trainable params: 80,376
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.07
Params size (MB): 0.31
Estimated Tot

In [ ]:
import torch.optim as optim


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(NewModel.parameters(), lr=0.01)


def train(model, epoch, device):
    model.train()
    
    for i in range(epoch):
        train_loss = 0
        correct = 0
        total = 0

        for idx, (input, target) in enumerate(trainloader):
            
            
            input = input.to(device)
            target = target.to(device)
            output = model(input)
            loss = criterion(output, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

        acc = 100 * correct / total
        print('train epoch : {} [{}/{}]| loss: {:.3f} | acc: {:.3f}'.format(
        i, idx, len(trainloader), train_loss/(idx+1), acc))
    
    print("end of training")
    

train(NewModel, 10, 'cuda')

train epoch : 0 [781/782]| loss: 2.303 | acc: 12.066
train epoch : 1 [781/782]| loss: 2.255 | acc: 15.902
train epoch : 2 [781/782]| loss: 2.032 | acc: 25.622
train epoch : 3 [781/782]| loss: 1.962 | acc: 28.754
train epoch : 4 [781/782]| loss: 1.857 | acc: 33.884
train epoch : 5 [781/782]| loss: 1.767 | acc: 37.626
train epoch : 6 [781/782]| loss: 1.705 | acc: 39.554
train epoch : 7 [781/782]| loss: 1.642 | acc: 41.626
train epoch : 8 [781/782]| loss: 1.585 | acc: 43.212
train epoch : 9 [781/782]| loss: 1.532 | acc: 45.254
end of training


DNN 보다 훨씬 더 가볍고 학습도 빠르면서 정확도도 높은 것을 확인해볼 수 있다.

conv를 더 촘촘히 쌓는다면 더더욱 높은 정확도를 얻을 수 있다.

이제 wk8 실습에서 직접 코딩을 해보도록 하자